In [2]:
import numpy as np
from astropy.timeseries import TimeSeries
import os
import matplotlib.pyplot as plt
import argparse
from astropy.time import Time, TimeDelta
from astropy import units as u
from IPython.core.display import display, HTML
import celerite
from celerite import terms
import corner
from scipy.optimize import minimize
import emcee
display(HTML("<style>.container { width:100% !important; }</style>"))
# format #Time    	T_+ve   	T_-ve   	Rate    	Ratepos 	Rateneg	HR      	HRerr   
#23908.258	23787.69	-23787.69	0.1172494	0.007794342	-0.007794342	2.942098	0.452518
def draw_arrows(x, y, color, ax=None):
    for i in np.arange(1, len(x)):
        if ax==None:
            plt.annotate("", xy=(x[i-1],y[i-1]), xytext=(x[i], y[i]), arrowprops=dict(arrowstyle="<-", shrinkA=10, shrinkB=10, color=color))
        else:
            ax.annotate("", xy=(x[i-1],y[i-1]), xytext=(x[i], y[i]), arrowprops=dict(arrowstyle="<-", shrinkA=10, shrinkB=10, color=color))


In [2]:
#ap = argparse.ArgumentParser(description='Clean line maps by thresholding them with the SNR maps')
#ap.add_argument("-b", "--binning", nargs='?', help="Binning in days for the plot. Default does not apply any binning (i.e. one observation per datapoint)", type=str, default="1")
#args = ap.parse_args()
#binning = args.binning
binning = 365 * 1 * 24 * 3600
# 2 weeks
time_threshold = 4 * 7 * 24 * 3600
home = os.getenv("HOME")
#in_dir = "%s/x_ray_data/holmbergIIX1/swift_data/USERPROD_90.76.134.147_1585306008859/lc" % home
in_dir = "%s/x_ray_data/NGC5204X-1/swift_data/USERPROD_90.76.134.147_1585759247813/lc" % home
#in_dir ="%s/x_ray_data/NGC1313X-1/swift_data/USERPROD_90.76.134.147_1585559104874/lc" % home
data = np.genfromtxt("%s/countrate_hr.txt" % in_dir, names=True, 
              delimiter="\t", dtype=("f8", "f8", "f8", "f8", "f8", "f8", "f8", "f8"))
plt.style.use('/home/agurpide/.config/matplotlib/stylelib/paper.mplstyle')

OSError: /home/agurpide/x_ray_data/NGC5204X-1/swift_data/USERPROD_90.76.134.147_1585759247813/lc/countrate_hr.txt not found.

In [3]:
%matplotlib qt
plt.xlabel("HR [1.5 - 10 keV] / [0.3 - 1.5 keV]")
plt.ylabel("Count rate (0.3 - 10 keV) s$^-1$")
if binning==1:
    plt.errorbar(data["HR"], data["Rate"], xerr=data["HRerr"], yerr=[-data["Rateneg"], data["Ratepos"]], ls="None", fmt="-", color="black")
else:
    init = data["Time"][0]
    end = data["Time"][-1]
    step = init
    
    while step < end:
        data_to_plot = data[np.where((data["Time"]>=step) & (data["Time"]<(step+binning)))]
        if len(data_to_plot["Time"]) == 0:
            step += binning
            continue
        plt.errorbar(data_to_plot["HR"], data_to_plot["Rate"], xerr=data_to_plot["HRerr"], yerr=[-data_to_plot["Rateneg"], data_to_plot["Ratepos"]], ls="None", fmt="-")
        step += binning
        
        print(step)
        

NameError: name 'data' is not defined

In [4]:
%matplotlib inline
plt.xlabel("HR [1.5 - 10 keV] / [0.3 - 1.5 keV]")
plt.ylabel("Count rate (0.3 - 10 keV) s$^-1$")
if binning==1:
    plt.errorbar(data["HR"], data["Rate"], xerr=data["HRerr"], yerr=[-data["Rateneg"], data["Ratepos"]], ls="None", fmt="-", color="black")
else:
    # create groups
    i = 0
    for time, j enumerate(data["Time"]):
        while (data["Time"][i] - data["Time"][i+1]) < time_thresold:
            i+=1
        plt.errorbar(data["HR"][j:i], data["Rate"][j:i], xerr=data["HRerr"][j:i],
                     yerr=[-data["Rateneg"][j:i], data["Ratepos"][j:i]], ls="None", marker=".", errorevery=5, markersize=5)
        
            
            # new group
            data_to_plot = data[np.where((data["Time"]>=step) & (data["Time"]<(step + binning)))[0]]
    # accumulate
    
    init = data["Time"][0]
    end = data["Time"][-1]
    step = init
    prop_cycle = plt.rcParams['axes.prop_cycle']
    colors = prop_cycle.by_key()['color']
    color_index = 0
    while step < end:
        data_to_plot = data[np.where((data["Time"]>=step) & (data["Time"]<(step + binning)))[0]]
        
        if len(data_to_plot["Time"]) < 5:
            step += binning
            continue
        plt.errorbar(data_to_plot["HR"], data_to_plot["Rate"], xerr=data_to_plot["HRerr"],
                     yerr=[-data_to_plot["Rateneg"], data_to_plot["Ratepos"]], ls="None", marker=".", errorevery=5, markersize=5)
        
        draw_arrows(data_to_plot["HR"], data_to_plot["Rate"], color=colors[color_index])
        plt.show()
        color_index +=1
        step += binning
    print(step)
    print(end)

SyntaxError: invalid syntax (<ipython-input-4-17ace8abd2da>, line 9)

In [8]:
%matplotlib qt
plt.xlabel("HR [1.5 - 10 keV] / [0.3 - 1.5 keV]")
plt.ylabel("Count rate (0.3 - 10 keV) s$^-1$")
min_points = 5
# get source distance
f = open("%s/../zero-point.txt" % in_dir)
lines = f.readlines()
f.close()
zero_point = float(lines[0])
# first of january of 2001 in mjdr + zero_poin to days
swift_ref_time = Time(51910 , format="mjd", scale="tt")
swift_zero_point = Time(swift_ref_time.mjd + zero_point / 24 / 3600, format="mjd", scale="tt")
print(swift_zero_point.mjd * 24 * 3600)

def time_todate(x):
    time = Time(x, format='mjd', scale="tt")
    time.format = "iso"
    time.out_subfmt = 'date'
    return time

if binning==1:
    plt.errorbar(data["HR"], data["Rate"], xerr=data["HRerr"], yerr=[-data["Rateneg"], data["Ratepos"]], ls="None", fmt="-", color="black")
else:
    # create groups
    i = 0
    j = 0
    chunk = 0
    prop_cycle = plt.rcParams['axes.prop_cycle']
    colors = prop_cycle.by_key()['color']
    while j < len(data["Time"]):
        if i==len(data["Time"]) - 1:
                print("End reached")
                break
        while ((abs(data["Time"][i] - data["Time"][i+1])) < time_threshold):   
            i += 1
            
            if i==len(data["Time"]) - 1:
                print("End reached")
                break
            
        accumulated_datapoints = (i-j + 1)
        if accumulated_datapoints < min_points:
            print("Less than %d datapoints (%d). Skipping" % (min_points, accumulated_datapoints))
            i += 1
            j = i
        else:
            print("%d datapoints" % accumulated_datapoints)
            plt.errorbar(data["HR"][j:i], data["Rate"][j:i], xerr=data["HRerr"][j:i],
                     yerr=[-data["Rateneg"][j:i], data["Ratepos"][j:i]], ls="None", marker=".", errorevery=8, markersize=8, elinewidth=0.5,
                         label="%d %.1f d (%d) %s" %(chunk+1, float(data["Time"][i]-data["Time"][j])/3600/24, accumulated_datapoints, time_todate(swift_zero_point.mjd + float(data["Time"][j])/3600/24).value))
            draw_arrows(data["HR"][j:i], data["Rate"][j:i], color=colors[chunk])
            i += 1
            j = i
            chunk += 1
    plt.legend()

4657573406.9
Less than 5 datapoints (1). Skipping
Less than 5 datapoints (2). Skipping
12 datapoints
17 datapoints
Less than 5 datapoints (2). Skipping
114 datapoints
70 datapoints
50 datapoints
20 datapoints
11 datapoints
25 datapoints
21 datapoints
9 datapoints
End reached
8 datapoints


IndexError: list index out of range

/home/agurpide/anaconda3/lib/python3.7/site-packages/ipykernel/eventloops.py:106: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations. 
  app.exec_()


In [9]:
f = open("%s/../zero-point.txt" % in_dir)
lines = f.readlines()
f.close()
zero_point = float(lines[0])
swift_ref_time = Time(51910 + zero_point/24/3600, format="mjd", scale="tt")
times = Time(swift_ref_time.mjd + data["Time"]/3600/24, format='mjd', scale="tt")
ts = TimeSeries({'countrate': data["Rate"] , 'error':((-data["Rateneg"] + data["Ratepos"]) / 2)}, time=times)
folded = ts.fold(28.5*u.day, wrap_phase=1)
print(folded)
plt.errorbar(folded.time.jd, folded['countrate'], yerr=folded["error"], ls="None", fmt="-", marker="o")

UnitsError: wrap_phase should be a Quantity in units of time when normalize_phase=False

In [18]:
print(zero_point)
%matplotlib qt
low_flux = np.array([row for row in data if row["Rate"] < 0.10])
low_flux = data
plt.errorbar(low_flux["Time"] / 3600 /24, low_flux["Rate"], yerr=[-low_flux["Rateneg"], low_flux["Ratepos"]], ecolor="black", 
             marker=".", ls="None")
plt.xlabel("Time(days)")
plt.ylabel("Count rate [0.3 - 10 keV] ct s$^{-1}$")
threshold_low = 0
#threshold_low = 0.17
#threshold_low = 0.17
#threshold_high = 10
threshold_high = 0.20
threshold_low = 0.12
#threshold_high= 0.10

plt.axhline(y=threshold_low, color="green", ls="--")
plt.axhline(y=threshold_high, color="green", ls="--")
j = 0
i = 0
string_out = ""
while j < len(data["Time"]):
    if i == (len(data["Time"]) - 1):
        print("End reached")
        break
    rate = data["Rate"][j]
    if rate > threshold_low and rate < threshold_high:
        print("Accumulating observations with rate below %.2f and above %.2f" % (threshold_high, threshold_low))
        i = j
        while rate > threshold_low and rate < threshold_high:
            if i == (len(data["Time"]) - 1):
                print("End reached")
                break
            i += 1
            rate = data["Rate"][i]
        
        print("Accumulated %d observations (%d - %d)" %( (i-j), data["Time"][j], data["Time"][i-1]))
        string_out+= "%.1f-%.1f," %(data["Time"][j], (data["Time"][i-1] + 0.5))
        i += 1
        j = i
    else:
        j += 1
        #print("Skipping observation with rate %.3f" % rate)
print(string_out)

172549406.9



/home/agurpide/anaconda3/lib/python3.7/site-packages/ipykernel/eventloops.py:106: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations. 
  app.exec_()


## Celerite fit

In [3]:
def readPCCURVE(file="PCCURVE.qdp"):
    """Read PCCURVE from Swift data pipeline.

        Parameters
        ----------
        file: str, optional
            The file to be read. Default is PCCURVE.qdp
        """
    print("Reading %s data" % file)
    return np.genfromtxt("%s" % file, names=True, delimiter="\t", skip_header=2, comments="!", dtype=("f8, f8, f8, f8, f8, f8, f8, f8, f8, f8, f8, f8, f8, f8, U30"))


def neg_log_like(params, y, gp):
    gp.set_parameter_vector(params)
    return -gp.log_likelihood(y)

def log_probability(params, y, gp):
    gp.set_parameter_vector(params)

    lp = gp.log_prior()
    if not np.isfinite(lp):
        return -np.inf

    ll = gp.log_likelihood(y)
    return ll + lp if np.isfinite(ll) else -np.inf

In [46]:
count_rate_file = "PCCURVE.qdp"
data = readPCCURVE("/home/agurpide/x_ray_data/NGC5204X-1/swift_data/lc/%s" % count_rate_file)
#data = np.array([row for row in data if row["Rate"] < 0.073])
def read_config_file(file):
    f = open(file)
    lines = f.readlines()
    f.close()
    param_info = lines[1].split("\t")
    return param_info

Reading /home/agurpide/x_ray_data/NGC5204X-1/swift_data/lc/PCCURVE.qdp data


In [51]:
freeze_omega = False
rate = "Rate"
y = data["%s" % rate]
time = data["Time"]
yerr = (-data["%sneg" % rate] + data["%spos" % rate]) / 2
params = read_config_file("/home/agurpide/scripts/swift_scripts/celerite_config/parameters.config")
w = np.log(2 * np.pi /  (np.array(params[0].split(":")).astype(float) * 24 * 3600))
S_0 = np.log(np.array(params[1].split(":")).astype(float))
Q = np.log(np.array(params[2].split(":")).astype(float))
bounds = dict(log_S0=(S_0[0], S_0[2]), log_Q=(Q[0], Q[2]), log_omega0=(w[2], w[0]))
kernel = terms.SHOTerm(log_S0=S_0[1], log_Q=Q[1], log_omega0=w[1], bounds=bounds)

if freeze_omega:
    kernel.freeze_parameter("log_omega0")
#kernel.freeze_parameter("log_Q")
gp = celerite.GP(kernel, mean=np.mean(data["Rate"]))
gp.compute(time, yerr)  # You always need to call compute once.
print("Initial log likelihood: {0}".format(gp.log_likelihood(y)))
initial_params = gp.get_parameter_vector()
bounds = gp.get_parameter_bounds()
par_names = gp.get_parameter_names()

Initial log likelihood: 184.7793390356785


## FIT

In [15]:
solution = minimize(neg_log_like, initial_params, method="L-BFGS-B", bounds=bounds, args=(y, gp))
gp.set_parameter_vector(solution.x)
print(solution)
params = gp.get_parameter_dict()
if not freeze_omega:
    print("Period = %.2f d" % (2 * np.pi / (np.exp(params["kernel:log_omega0"]) * 3600 * 24)))
print("Q factor = %.2f" % np.exp(params["kernel:log_Q"]))
%matplotlib qt
figure, (ax1, ax2) = plt.subplots(2 ,1, sharex=True)
samples = np.linspace(np.min(time), np.max(time), 5000)
pred_mean, pred_var = gp.predict(y, samples, return_var=True)
pred_std = np.sqrt(pred_var)
color = "#ff7f0e"
ax1.errorbar(time, y, yerr=yerr, fmt=".k", capsize=0)
ax1.plot(samples, pred_mean, color=color)
ax1.fill_between(samples, pred_mean+pred_std, pred_mean-pred_std, color=color, alpha=0.3,
                 edgecolor="none")
pred_mean, pred_var = gp.predict(y, time, return_var=True)
ax2.errorbar(time, (y - pred_mean) / yerr, yerr=1, fmt=".k", capsize=0)
ax2.axhline(y=0, ls="--", color="#afdbf5")
ax2.set_ylabel("Residuals")

      fun: -200.92382301146324
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.        ,  1.1146426 , -4.83612439])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 48
      nit: 9
   status: 0
  success: True
        x: array([  4.36722309,   1.        , -14.53950118])
Period = 150.00 d
Q factor = 2.72


Text(0, 0.5, 'Residuals')

## MCMC

(20, 3)
(3,)
[[  3.2165999    4.09476481 -14.3903983 ]
 [  3.2165999    4.09476481 -14.3903983 ]
 [  3.2165999    4.09476481 -14.3903983 ]
 [  3.2165999    4.09476481 -14.3903983 ]
 [  3.2165999    4.09476481 -14.3903983 ]
 [  3.2165999    4.09476481 -14.3903983 ]
 [  3.2165999    4.09476481 -14.3903983 ]
 [  3.2165999    4.09476481 -14.3903983 ]
 [  3.2165999    4.09476481 -14.3903983 ]
 [  3.2165999    4.09476481 -14.3903983 ]
 [  3.2165999    4.09476481 -14.3903983 ]
 [  3.2165999    4.09476481 -14.3903983 ]
 [  3.2165999    4.09476481 -14.3903983 ]
 [  3.2165999    4.09476481 -14.3903983 ]
 [  3.2165999    4.09476481 -14.3903983 ]
 [  3.2165999    4.09476481 -14.3903983 ]
 [  3.2165999    4.09476481 -14.3903983 ]
 [  3.2165999    4.09476481 -14.3903983 ]
 [  3.2165999    4.09476481 -14.3903983 ]
 [  3.2165999    4.09476481 -14.3903983 ]]


In [81]:
init_params = gp.get_parameter_vector()
ndim = len(init_params)
# set from the example
nwalkers = 20
# start from the best fit values
initial_samples = gp.get_parameter_vector() + 1e-5 * np.random.randn(nwalkers, ndim)
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=(y, gp))
# burn in phase
sampler.run_mcmc(initial_samples, 5000, progress="notebook")
#final_samples = sampler.get_chain(discard=500, thin=50, flat=True)
acceptance_ratio = sampler.acceptance_fraction
print(acceptance_ratio)
tau = sampler.get_autocorr_time()
print(tau)


[0.5648 0.553  0.5458 0.5478 0.533  0.5772 0.5274 0.5382 0.5682 0.5602
 0.5592 0.5604 0.5428 0.5382 0.562  0.5506 0.5454 0.537  0.5718 0.5638]
[51.41441177 66.19463011 51.7345795 ]


In [22]:
%matplotlib qt
chain = sampler.get_chain(flat=True)
fig, axes = plt.subplots(3, sharex=True, gridspec_kw={'hspace': 0, 'wspace': 0})
means = np.mean(chain, axis=0)
for param, parname, ax, mean in zip(chain.T, par_names, axes, means):
    ax.plot(param, linestyle="None", marker="+", color="black")
    ax.set_ylabel(parname)
    ax.axhline(y=mean)
# This suggests that only about 40 steps are needed for the chain to “forget” where it started. It’s not unreasonable to throw away a few times 
# this number of steps as “burn-in”. Let’s discard the initial 100 steps, thin by about half the autocorrelation time (15 steps), 
# and flatten the chain so that we have a flat list of samples:

In [23]:
discard = 1000
for chain, parname, ax, mean in zip(chain.T, par_names, axes, means):
    ax.axvline(discard * nwalkers, ls="--", color="red")
final_samples = sampler.get_chain(discard=discard, thin=20, flat=True)

In [24]:
# transform names to human
names = list(gp.get_parameter_names())
cols = ["log_omega0", "log_S0", "log_Q"]
inds = [names.index("kernel:%s" %c) for c in cols]
samples = final_samples[:, inds]
samples[:, :-1] = np.exp(samples[:, :-1])
2 * np.pi / (np.exp(gp.get_parameter_dict()["kernel:log_omega0"]) * 3600 * 24)
ind_omega = names.index("kernel:%s" % "log_omega0")
samples[:,0] = 2 * np.pi / (samples[:,0]  * 3600 * 24)
medians = np.median(samples, axis=0)
ranges = [(median - 0.9 * median, median + 0.9 * median) for median in medians]
corner_fig = corner.corner(samples, smooth=0.5, range=ranges,labels=[r"$P$ (days)", r"$S_0$", r"Q"], truths=medians)
plt.show()

## Plot PSD and Model 

In [25]:
# plot 24 different results
model_figure, model_ax = plt.subplots()
model_ax.set_xlabel("Days")
model_ax.set_ylabel("Count-rate ct/s [0.3 - 10 keV]")
psd_figure, psd_ax = plt.subplots()
psd_ax.set_xlabel("Frequency [Days$^{-1}$]")
t_samples = 10000
t = np.linspace(np.min(time), np.max(time), t_samples)
min_f = np.exp(min_w)
max_f = np.exp(max_w)
frequencies = np.linspace(min_f, max_f, 5000)
color="orange"
n_samples = 1000
models = np.empty((n_samples, t_samples))
for index, sample in enumerate(final_samples[np.random.randint(len(samples), size=n_samples)]):
    gp.set_parameter_vector(sample)
    psd = gp.kernel.get_psd(frequencies)
    model = gp.predict(y, t, return_cov=False)
    model_ax.plot(t  / 3600 / 24 , model, color=color, alpha=0.3)
    psd_ax.plot(frequencies * 3600 * 24 / 2 / np.pi, psd, color="blue", alpha=0.3)
    models[index] = model
median_model = np.percentile(models, 50, axis=0)
#model_ax.plot(t  / 3600 / 24 , median_model, color=color)
model_ax.errorbar(time / 3600 / 24, y, yerr=yerr, fmt=".k", capsize=0)

<ErrorbarContainer object of 3 artists>

In [25]:
# plot 24 different results
model_figure, model_ax = plt.subplots()
model_ax.set_xlabel("Days")
model_ax.set_ylabel("Count-rate ct/s [0.3 - 10 keV]")
psd_figure, psd_ax = plt.subplots()
psd_ax.set_xlabel("Frequency [Days$^{-1}$]")
t_samples = 10000
t = np.linspace(np.min(time), np.max(time), t_samples)
min_f = np.exp(min_w)
max_f = np.exp(max_w)
frequencies = np.linspace(min_f, max_f, 1000)
color="orange"
n_samples = 1000
psds = np.empty((n_samples, len(frequencies)))
models = np.empty((n_samples, t_samples))
for index, sample in enumerate(final_samples[np.random.randint(len(samples), size=n_samples)]):
    gp.set_parameter_vector(sample)
    models[index] = gp.predict(y, t, return_cov=False)
    psds[index] = gp.kernel.get_psd(frequencies)

p = np.percentile(psds, [32, 50, 68], axis=0)
m = np.percentile(models, [32, 50, 68], axis=0)
model_ax.plot(t  / 3600 / 24 , m[1], color=color)
model_ax.fill_between(t / 3600 / 24, m[0], m[2], alpha=0.3, color=color)
model_ax.errorbar(time / 3600 / 24, y, yerr=yerr, fmt=".k", capsize=0)
psd_ax.plot(frequencies * 3600 * 24 / 2 / np.pi, p[1], color="blue")
psd_ax.fill_between(frequencies * 3600 * 24 / 2 / np.pi, p[0], p[2], color="blue", alpha=0.3)